In [182]:
import pandas as pd
!pip install -r requirements.txt

Obtaining networkx from git+https://github.com/antonio-turco/networkx@e0634ff8d8b555572fa396bf41bb9eddf85c8065#egg=networkx (from -r requirements.txt (line 113))
  Skipping because already up-to-date.


  Attempting uninstall: networkx
    Found existing installation: networkx 2.5rc1.dev20200607134054
    Uninstalling networkx-2.5rc1.dev20200607134054:
      Successfully uninstalled networkx-2.5rc1.dev20200607134054
  Running setup.py develop for networkx


In [184]:
filepath = "mushroom_data_all.csv"
mushroom_data = pd.read_csv(filepath)

mushroom_data.describe()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [185]:
mushroom_data.head()

,class_edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [186]:
print(mushroom_data.shape)
test_size = 1000
test_data = mushroom_data[-test_size:]
mushroom_data = mushroom_data[:-test_size]

y_test = test_data.class_edible
X_test = test_data.drop(labels = 'class_edible', axis=1)

print(mushroom_data.shape, test_data.shape)

(8124, 23)
(7124, 23) (1000, 23)


In [187]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=3)

In [188]:
small_cols = []

threshold = 10
for column in X_train.columns:
    if len(mushroom_data[column].unique()) < threshold:
        small_cols.append(column)
        
large_cols = list(set(X_train.columns) - set(small_cols))
        
print('Columns that will be encoded with One Hot Encoder: ', small_cols, end = '\n\n')
print('Columns that will be encoded with Label Encoder', large_cols)

Columns that will be encoded with One Hot Encoder:  ['cap-shape', 'cap-surface', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']

Columns that will be encoded with Label Encoder ['cap-color', 'gill-color']


In [189]:
from sklearn.preprocessing import LabelEncoder 
label_encoder = LabelEncoder()

from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [190]:
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
label_X_test = X_test.copy()

for col in large_cols:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = label_encoder.transform(label_X_valid[col])
    label_X_test[col] = label_encoder.transform(label_X_test[col])
    
OH_X_train = pd.DataFrame(OH_encoder.fit_transform(label_X_train[small_cols]))
OH_X_valid = pd.DataFrame(OH_encoder.transform(label_X_valid[small_cols]))
OH_X_test = pd.DataFrame(OH_encoder.transform(label_X_test[small_cols]))

OH_X_train.index = label_X_train.index
OH_X_valid.index = label_X_valid.index
OH_X_test.index = label_X_test.index


label_X_train = label_X_train.drop(small_cols, axis=1)
label_X_valid = label_X_valid.drop(small_cols, axis=1)
label_X_test = label_X_test.drop(small_cols, axis=1)

clean_X_train = pd.concat([OH_X_train, label_X_train], axis=1)
clean_X_valid = pd.concat([OH_X_valid, label_X_valid], axis=1)
clean_X_test = pd.concat([OH_X_test, label_X_test], axis=1)

clean_X_train

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,cap-color,gill-color
5788,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,10
414,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4,4
1011,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8,4
7958,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8,2
2664,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,0
1667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8,10
3321,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,10
1688,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,7


In [191]:
label_y_train = y_train.copy()
label_y_train = pd.Series(label_encoder.fit_transform(label_y_train), index = label_y_train.index)

label_y_valid = y_valid.copy()
label_y_valid = pd.Series(label_encoder.transform(label_y_valid), index = label_y_valid.index)

label_y_test = y_test.copy()
label_y_test = pd.Series(label_encoder.transform(label_y_test), index = label_y_test.index)

label_y_train.head(), y_train.head()

(5788    1
 414     1
 1011    0
 7958    0
 2664    0
 dtype: int64,
 5788    p
 414     p
 1011    e
 7958    e
 2664    e
 Name: class_edible, dtype: object)

In [192]:
from sklearn.ensemble import RandomForestClassifier

estimators = [50,75,100,125,150,200]
depths = [3,4,5,10,20]

tests = [ (estimator, depth) for estimator in estimators for depth in depths]

for test in tests:
    estimator, depth = test
    model = RandomForestClassifier(random_state = 3, n_estimators = estimator, max_depth = depth)
    model.fit(clean_X_train, label_y_train)
    score = model.score(clean_X_valid, label_y_valid)
    print('With test: ', test, 'score: ', score)

With test:  (50, 3) score:  0.9876923076923076
With test:  (50, 4) score:  0.9932307692307693
With test:  (50, 5) score:  0.9956923076923077
With test:  (50, 10) score:  1.0
With test:  (50, 20) score:  1.0
With test:  (75, 3) score:  0.9901538461538462
With test:  (75, 4) score:  0.9932307692307693
With test:  (75, 5) score:  0.9932307692307693
With test:  (75, 10) score:  1.0
With test:  (75, 20) score:  1.0
With test:  (100, 3) score:  0.9876923076923076
With test:  (100, 4) score:  0.9932307692307693
With test:  (100, 5) score:  0.9932307692307693
With test:  (100, 10) score:  1.0
With test:  (100, 20) score:  1.0
With test:  (125, 3) score:  0.9883076923076923
With test:  (125, 4) score:  0.9932307692307693
With test:  (125, 5) score:  0.9938461538461538
With test:  (125, 10) score:  1.0
With test:  (125, 20) score:  1.0
With test:  (150, 3) score:  0.9907692307692307
With test:  (150, 4) score:  0.9932307692307693
With test:  (150, 5) score:  0.9938461538461538
With test:  (150, 

Tra i vari parametri scarto quelli con un training score troppo elevato poichè probabilmente potrebbe esserci overfitting. Visto che nella maggior parte dei casi in cui si ottengono valori come il 100% di accuracy la profondità dell'albero è troppo elevata. Quindi si è scelta una via di mezzo per cercare di avere un modello che abbia un comportamento corretto anche in produzione

In [193]:
model_selected = RandomForestClassifier(random_state = 3, n_estimators = 150, max_depth = 5)
model_selected.fit(clean_X_train, label_y_train)
model_selected.score(clean_X_test, label_y_test)

1.0